<a href="https://colab.research.google.com/github/UiinKim/NLP_First/blob/main/2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

In [4]:
class RNN:
  def __init(self, Wx, Wh, b):
    self.params=[Wx, Wh, b] #파라미터는 x의 가중치, h의 가중치, 편향
    self.grads=[np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] #기울기는 0으로 초기화
    self.cache=None

  def forward(self, x, h_prev): #input인 x와 이전 단어의 h(은닉상태)
    Wx, Wh, b=self.params
    t=np.matmul(h_prev, Wh)+np.matmul(x, Wx)+b #이전 은닉상태와 가중치 + 현재 input과 가중치 + 편향 -> ht
    h_next=np.tanh(t) #하이퍼볼릭 탄젠트로 은닉 상태 완료

    self.cache=(x, h_prev, h_next) # t시점의 input, t-1시점의 h, t 시점의 h 저장
    return h_next #t시점의 h 출력

  def backward(self, dh_next):
    Wx, Wh, b = self.params
    x, h_prev, h_next = self.cache

    dt=dh_next*(1-h_next**2) #하이퍼볼릭 탄젠트에 대한 복구
    db=np.sum(dt, axis=0) #편향 기울기
    dWh=np.matmul(h_prev.T, dt) #이전 h의 가중치에 대한 기울기
    dh_prev=np.matmul(dt, Wh.T) #이전 h에 대한 기울기
    dWx=np.matmul(x.T, dt) #input의 가중치에 대한 기울기
    dx=np.matmul(dt, Wx.T) #input에 대한 기울기

    self.grads[0][...] =dWx #기울기에 각 정보 저장
    self.grads[1][...]= dWh
    self.grads[2][...] = db

    return dx, dh_prev


In [5]:
class TimeRNN:
  def __init__(self, Wx, Wh, b, stateful=False): #stateful은 앞의 은닉상태가 있나 없나 확인
    self.params=[Wx, Wh, b]
    self.grads=[np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
    self.layers=None

    self.h, self.dh=None, None
    self.stateful=stateful

  def set_state(self, h):
    self.h=h

  def reset_state(self):
    self.h=None

  def forward(self, xs): #xs의 shape는 (N, T, D)
    Wx, Wh, b = self.params #가중치와 편향 그대로 받아오기
    N, T, D = xs.shape #N은 미니배치 크기, T는 개수, D는 input vector의 차원 수
    D, H = Wx.shape

    self.layers=[]
    hs=np.empty((N, T, H), dtype='f') #hs는 output으로 나올 h

    if not self.stateful or self.h is None:
      self.h = np.zeros((N, H), dtype='f') #stateful=False이거나 저장된 h가 없으면 0으로 초기화

    for t in range(T): #0~T까지 모든 벡터 동안
      layer=RNN(*self.params) #*는 리스트의 원소(Wx, Wh, b)들을 추출하여 RNN의 인수로 전달
      self.h=layer.forward(xs[:,t,:], self.h) #xs의 미니배치 별로 , 해당 t시점부분, 벡터 전부를 RNN 클래스의 순전파 시켜 현 시점의 은닉상태로
      hs[:,t,:]=self.h #hs에 그대로 전달
      self.layers.append(layer) #새로운 층으로 추가

    return hs #hs 반환

  def backward(self, dhs):
    Wx, Wh, b=self.params
    N, T, H = dhs.shape #역전파되는 dhs의 미니배치, 벡터 개수, 은닉 차원
    D, H = Wx.shape

    dxs=np.empty((N, T, D), dtype='f') #원래의 xs의 shape으로 만들기
    dh=0
    grads=[0,0,0]
    for t in reversed(range(T)): #T-1시점부터 역전파 시작
      layer=self.layers[t] #t시점의 층으로 설정
      dx, dh=layer.backward(dhs[:,t,:]+dh) #역전파를 통해 x의 기울기와 h의 기울기 얻기
      dxs[:,t,:]=dx #dxs에 구한 x의 기울기 넣기

      for i, grad in enumerate(layer.grads): #해당 층의 기울기 업데이트
        grads[i]+=grad

    for i, grad in enumerate(grads): #해당 층의 기울기 업데이트
      self.grads[i][...]=grad
    self.dh=dh #h의 기울기 저장

    return dxs #x의 기울기 반환


